In [1]:
import sympy

We are looking to predict the evolution of mixed invariant vector fields exactly:
    
$\dot{X} = r X + X l$

Which has the solution:
    
$X(t) = e^{rt} X(0) e^{lt}$

However, r and l are not necessary in so(3). It is required that $Ad_X r + l$ and $Ad_X l + r$ are in so(3).

We can easily find the exponential for the right sided vector field for the mixed invariant system. The exponential of this, will not be a group element due to element (3, 4) becoming 1. However, this is cancelled by the $l$ matrix of the mixed-invariant system, and the evolution remains on the manifold.

In [2]:
g = sympy.symbols("g")
r = sympy.Matrix(
    [
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, g, 0],
        [0, 0, 0, 0, -1],
        [0, 0, 0, 0, 0],
    ]
)
sympy.exp(r)

Matrix([
[1, 0, 0, 0,    0],
[0, 1, 0, 0,    0],
[0, 0, 1, g, -g/2],
[0, 0, 0, 1,   -1],
[0, 0, 0, 0,    1]])

The $l$ matrix is not trivial to find a closed form exponential for.

In [3]:
omega_x, omega_y, omega_z, a_x, a_y, a_z = sympy.symbols(
    "omega_x, omega_y, omega_z, a_x, a_y, a_z"
)
l = sympy.Matrix(
    [
        [0, -omega_z, omega_y, 0, 0],
        [omega_z, 0, -omega_x, 0, 0],
        [-omega_y, omega_x, 0, 0, 0],
        [0, 0, 0, 0, -1],
        [0, 0, 0, 0, 0],
    ]
)
l

Matrix([
[       0, -omega_z,  omega_y, 0,  0],
[ omega_z,        0, -omega_x, 0,  0],
[-omega_y,  omega_x,        0, 0,  0],
[       0,        0,        0, 0, -1],
[       0,        0,        0, 0,  0]])

We will analyze this matrix using block matrices. Notice that B is nilpotent. $B^2=0$, $B^3=0$.

In [4]:
A = sympy.MatrixSymbol("A", 3, 2)
B = sympy.MatrixSymbol("B", 2, 2)
Omega = sympy.MatrixSymbol("Omega", 3, 3)
S = sympy.BlockMatrix([[Omega, A], [sympy.ZeroMatrix(2, 3), B]])
S

Matrix([
[Omega, A],
[    0, B]])

We can solve this, since B is nilpotent. To spot the series pattern, we will print several terms.

In [5]:
def find_term(n):
    sub_B_nilpotent = {B**k: sympy.ZeroMatrix(2, 2) for k in range(2, n + 1)}
    return sympy.block_collapse(
        sympy.block_collapse(S**n / sympy.factorial(n)).expand().subs(sub_B_nilpotent)
    )

In [6]:
find_term(1)

Matrix([
[Omega, A],
[    0, B]])

In [7]:
find_term(2)

Matrix([
[(1/2)*Omega**2, (1/2)*A*B + (1/2)*Omega*A],
[             0,                         0]])

In [8]:
find_term(3)

Matrix([
[(1/6)*Omega**3, (1/6)*Omega**2*A + (1/6)*Omega*A*B],
[             0,                                  0]])

In [9]:
find_term(4)

Matrix([
[(1/24)*Omega**4, (1/24)*Omega**2*A*B + (1/24)*Omega**3*A],
[              0,                                       0]])

In [10]:
T = sympy.ZeroMatrix(5, 5)
for i in range(5):
    T += find_term(i)
T = sympy.block_collapse(T)
T

Matrix([
[I + (1/2)*Omega**2 + (1/6)*Omega**3 + (1/24)*Omega**4 + Omega, (1/6)*Omega**2*A + (1/24)*Omega**2*A*B + (1/24)*Omega**3*A + A + (1/2)*A*B + (1/2)*Omega*A + (1/6)*Omega*A*B],
[                                                            0,                                                                                                        I + B]])

We now create a series for the top right term.

In [11]:
n = sympy.symbols("n", integer=True)
k = sympy.symbols("k", integer=True)
theta = sympy.symbols("theta")

expr = (Omega ** (k - 1) * A + Omega ** (k - 2) * A * B) / sympy.factorial(k)
expr

1/factorial(k)*(Omega**(k - 2)*A*B + Omega**(k - 1)*A)

The series is only valid after k=2

In [12]:
sympy.block_collapse(expr.subs(k, 2))

(1/2)*A*B + (1/2)*Omega*A

In [13]:
sympy.block_collapse(expr.subs(k, 3))

(1/6)*Omega**2*A + (1/6)*Omega*A*B

In [14]:
sympy.block_collapse(expr.subs(k, 4))

(1/24)*Omega**2*A*B + (1/24)*Omega**3*A

$ l = \begin{bmatrix} \Omega && A \\ 0 && B \end{bmatrix} $
